In [1]:
'''
    This is based on the micasense radiometric calibration tutorial
    available at https://github.com/micasense/imageprocessing and MO Danilevicz Github user
'''

#load_ext autoreload
#autoreload 2
from ipywidgets import FloatProgress, Layout
from IPython.display import display
import micasense.imageset as imageset
import micasense.capture as capture
import os, glob
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib as plt
import subprocess
import argparse
import datetime

panelNames = None
useDLS = True


imagePath = os.path.expanduser(os.path.join('\\\\ds3617xs1\\Sensorik\\Projektdaten\\COBRI\\2020\\1_SOURCE_DATA\\UAV\\IFZ\\Multispectral\\2020_07_28\\raw'))
panelPath= '\\\\ds3617xs1\\Sensorik\\Projektdaten\\COBRI\\2020\\1_SOURCE_DATA\\UAV\\IFZ\\Multispectral\\2020_07_28\\panel'

outputPath = '\\\\ds3617xs1\\Sensorik\\Projektdaten\\COBRI\\2020\\1_SOURCE_DATA\\UAV\\IFZ\\Multispectral\\2020_07_28\\stacks'
thumbnailPath = os.path.join(outputPath, '..', 'thumbnails')

overwrite = False # can be set to set to False to continue interrupted processing
generateThumbnails = True
##################################################################################
#################################################################################
# Allow this code to align both radiance and reflectance images; bu excluding
# a definition for panelNames above, radiance images will be used
# For panel images, efforts will be made to automatically extract the panel information
# but if the panel/firmware is before Altum 1.3.5, RedEdge 5.1.7 the panel reflectance
# will need to be set in the panel_reflectance_by_band variable.
# Note: radiance images will not be used to properly create NDVI/NDRE images below.

    # Create Panel Imageset
panelset = imageset.ImageSet.from_directory(panelPath)
panelCap = panelset.captures
irradiances = []
for capture in panelCap:
        if capture.panel_albedo() is not None and not any(v is None for v in capture.panel_albedo()):
            panel_reflectance_by_band = capture.panel_albedo()
            panel_irradiance = capture.panel_irradiance(panel_reflectance_by_band)
            irradiances.append(panel_irradiance)
        img_type='reflectance'
    # Get the mean reflectance per band considering all panel images
df_panel = pd.DataFrame(irradiances)
mean_irradiance = df_panel.mean(axis=0)
mean_irradiance = mean_irradiance.values.tolist()

## This progress widget is used for display of the long-running process
f = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Loading")
display(f)
def update_f(val):
    if (val - f.value) > 0.005 or val == 1: #reduces cpu usage from updating the progressbar by 10x
        f.value=val

imgset = imageset.ImageSet.from_directory(imagePath, progress_callback=update_f)
update_f(1.0)

import math
import numpy as np
from mapboxgl.viz import *
from mapboxgl.utils import df_to_geojson, create_radius_stops, scale_between
from mapboxgl.utils import create_color_stops
import pandas as pd

data, columns = imgset.as_nested_lists()
df = pd.DataFrame.from_records(data, index='timestamp', columns=columns)

#Insert your mapbox token here
token = 'pk.eyJ1IjoibWljYXNlbnNlIiwiYSI6ImNqYWx5dWNteTJ3cWYzMnBicmZid3g2YzcifQ.Zrq9t7GYocBtBzYyT3P4sw'
color_property = 'dls-yaw'
num_color_classes = 8

min_val = df[color_property].min()
max_val = df[color_property].max()

import jenkspy
breaks = jenkspy.jenks_breaks(df[color_property], nb_class=num_color_classes)

color_stops = create_color_stops(breaks,colors='YlOrRd')
geojson_data = df_to_geojson(df,columns[3:],lat='latitude',lon='longitude')

viz = CircleViz(geojson_data, access_token=token, color_property=color_property,
                color_stops=color_stops,
                center=[df['longitude'].median(),df['latitude'].median()], 
                zoom=16, height='600px',
                style='mapbox://styles/mapbox/satellite-streets-v9')
viz.show()

from numpy import array
from numpy import float32
import cv2
import micasense.imageutils as imageutils
# Set warp_matrices to none to align using RigRelatives
# Or
# Use the warp_matrices derived from the Alignment Tutorial for this RedEdge set without RigRelatives
# Imageset transforms

    # Imageset transforms
    # Alignment settings
match_index = 1 # Index of the band I will try to match all others
max_alignment_iterations = 30 #increase max_iterations for better results, but longer runtimes
warp_mode = cv2.MOTION_HOMOGRAPHY # for Altum images only use HOMOGRAPHY
pyramid_levels =1 # for images with Rigrelatives, setting this to 0 or 1 may improve the alignment
    
    ## Find the warp_matrices for one of the images
    #chose a random middle of the flight capture (like 50?)
    
matrice_sample = imgset.captures[round(len(imgset.captures)/2)]
warp_matrices, alignment_pairs = imageutils.align_capture(matrice_sample,
                                                          ref_index = match_index,
                                                          max_iterations = max_alignment_iterations,
                                                          warp_mode = warp_mode,
                                                          pyramid_levels = pyramid_levels)
print("Finished Aligning, warp matrices={}".format(warp_matrices))
    
###Plotting aligtment test (optional)
###########################################################################################################
# cropped_dimensions, edges = imageutils.find_crop_bounds(capture, warp_matrices, warp_mode=warp_mode)
# im_aligned = imageutils.aligned_capture(capture, warp_matrices, warp_mode, cropped_dimensions, match_index, img_type=img_type)

# figsize=(30,23) # use this size for full-image-resolution display
# # figsize=(16,13)   # use this size for export-sized display

# rgb_band_indices = [capture.band_names_lower().index('red'),
#                     capture.band_names_lower().index('green'),
#                     capture.band_names_lower().index('blue')]
# cir_band_indices = [capture.band_names_lower().index('nir'),
#                     capture.band_names_lower().index('red'),
#                     capture.band_names_lower().index('green')]

# # Create a normalized stack for viewing
# im_display = np.zeros((im_aligned.shape[0],im_aligned.shape[1],im_aligned.shape[2]), dtype=np.float32 )

# im_min = np.percentile(im_aligned[:,:,rgb_band_indices].flatten(), 0.5)  # modify these percentiles to adjust contrast
# im_max = np.percentile(im_aligned[:,:,rgb_band_indices].flatten(), 99.5)  # for many images, 0.5 and 99.5 are good values

# # for rgb true color, we use the same min and max scaling across the 3 bands to 
# # maintain the "white balance" of the calibrated image
# for i in rgb_band_indices:
#     im_display[:,:,i] =  imageutils.normalize(im_aligned[:,:,i], im_min, im_max)

# rgb = im_display[:,:,rgb_band_indices]

# # for cir false color imagery, we normalize the NIR,R,G bands within themselves, which provides
# # the classical CIR rendering where plants are red and soil takes on a blue tint
# for i in cir_band_indices:
#     im_display[:,:,i] =  imageutils.normalize(im_aligned[:,:,i])

# cir = im_display[:,:,cir_band_indices]
# fig, axes = plt.subplots(1, 2, figsize=figsize)
# axes[0].set_title("Red-Green-Blue Composite")
# axes[0].imshow(rgb)
# axes[1].set_title("Color Infrared (CIR) Composite")
# axes[1].imshow(cir)
# plt.show()
########################################################################################################    
# save warp_matrices used for the imgset alignment
with open(os.path.join(outputPath, 'warp_matrices.txt'),'w') as f:
        f.write(str(warp_matrices))

import exiftool
import datetime
## This progress widget is used for display of the long-running process
f2 = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Saving")
display(f2)
def update_f2(val):
    f2.value=val

if not os.path.exists(outputPath):
    os.makedirs(outputPath)
if generateThumbnails and not os.path.exists(thumbnailPath):
    os.makedirs(thumbnailPath)

# Save out geojson data so we can open the image capture locations in our GIS
with open(os.path.join(outputPath,'imageSet.json'),'w') as f:
    f.write(str(geojson_data))
    
try:
    irradiance = panel_irradiance+[0]
except NameError:
    irradiance = None

start = datetime.datetime.now()
for i,capture in enumerate(imgset.captures):
    outputFilename = capture.uuid+'.tif'
    thumbnailFilename = capture.uuid+'.jpg'
    fullOutputPath = os.path.join(outputPath, outputFilename)
    fullThumbnailPath= os.path.join(thumbnailPath, thumbnailFilename)
    if (not os.path.exists(fullOutputPath)) or overwrite:
        if(len(capture.images) == len(imgset.captures[0].images)):
            capture.create_aligned_capture(irradiance_list=irradiance, warp_matrices=warp_matrices)
            capture.save_capture_as_stack(fullOutputPath)
            if generateThumbnails:
                capture.save_capture_as_rgb(fullThumbnailPath)
    capture.clear_image_data()
    update_f2(float(i)/float(len(imgset.captures)))
update_f2(1.0)
end = datetime.datetime.now()

print("Saving time: {}".format(end-start))
print("Alignment+Saving rate: {:.2f} images per second".format(float(len(imgset.captures))/float((end-start).total_seconds())))
######################################################

def decdeg2dms(dd):
   is_positive = dd >= 0
   dd = abs(dd)
   minutes,seconds = divmod(dd*3600,60)
   degrees,minutes = divmod(minutes,60)
   degrees = degrees if is_positive else -degrees
   return (degrees,minutes,seconds)

header = "SourceFile,\
GPSDateStamp,GPSTimeStamp,\
GPSLatitude,GpsLatitudeRef,\
GPSLongitude,GPSLongitudeRef,\
GPSAltitude,GPSAltitudeRef,\
FocalLength,\
XResolution,YResolution,ResolutionUnits\n"

lines = [header]
for capture in imgset.captures:
    #get lat,lon,alt,time
    outputFilename = capture.uuid+'.tif'
    fullOutputPath = os.path.join(outputPath, outputFilename)
    lat,lon,alt = capture.location()
    #write to csv in format:
    # IMG_0199_1.tif,"33 deg 32' 9.73"" N","111 deg 51' 1.41"" W",526 m Above Sea Level
    latdeg, latmin, latsec = decdeg2dms(lat)
    londeg, lonmin, lonsec = decdeg2dms(lon)
    latdir = 'North'
    if latdeg < 0:
        latdeg = -latdeg
        latdir = 'South'
    londir = 'East'
    if londeg < 0:
        londeg = -londeg
        londir = 'West'
    resolution = capture.images[0].focal_plane_resolution_px_per_mm

    linestr = '"{}",'.format(fullOutputPath)
    linestr += capture.utc_time().strftime("%Y:%m:%d,%H:%M:%S,")
    linestr += '"{:d} deg {:d}\' {:.2f}"" {}",{},'.format(int(latdeg),int(latmin),latsec,latdir[0],latdir)
    linestr += '"{:d} deg {:d}\' {:.2f}"" {}",{},{:.1f} m Above Sea Level,Above Sea Level,'.format(int(londeg),int(lonmin),lonsec,londir[0],londir,alt)
    linestr += '{}'.format(capture.images[0].focal_length)
    linestr += '{},{},mm'.format(resolution,resolution)
    linestr += '\n' # when writing in text mode, the write command will convert to os.linesep
    lines.append(linestr)

fullCsvPath = os.path.join(outputPath,'log.csv')
with open(fullCsvPath, 'w') as csvfile: #create CSV
    csvfile.writelines(lines)
	
import subprocess

if os.environ.get('exiftoolpath') is not None:
    exiftool_cmd = os.path.normpath(os.environ.get('exiftoolpath'))
else:
    exiftool_cmd = 'exiftool'
        
cmd = '{} -csv="{}" -overwrite_original {}'.format(exiftool_cmd, fullCsvPath, outputPath)
print(cmd)
subprocess.check_call(cmd)



FloatProgress(value=0.0, description='Loading', layout=Layout(width='100%'), max=1.0)

C:\Users\Barreto\Anaconda3\envs\micasense\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Finished aligning band 1
Finished aligning band 0
Finished aligning band 3
Finished aligning band 2
Finished aligning band 4
Finished Aligning, warp matrices=[array([[ 1.0012569e+00,  8.9975062e-04, -7.8356701e-01],
       [-1.8506713e-03,  1.0021272e+00,  1.7082647e+01],
       [-7.3640777e-08,  1.4570408e-07,  1.0000000e+00]], dtype=float32), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32), array([[ 9.9885899e-01,  6.3050241e-04,  7.9167714e+00],
       [-9.5582369e-04,  9.9947500e-01,  1.3326410e+01],
       [-7.8772246e-07,  5.1360450e-07,  1.0000000e+00]], dtype=float32), array([[ 9.9609965e-01, -2.0747480e-03,  3.0599055e+00],
       [ 4.2506226e-04,  9.9848151e-01, -1.3675878e+01],
       [-1.4538419e-06,  4.8817208e-07,  1.0000000e+00]], dtype=float32), array([[ 9.9723673e-01, -3.4122432e-03,  2.6870978e+00],
       [ 2.0750880e-03,  9.9972779e-01,  2.5661192e+00],
       [-1.5248035e-06,  3.2296319e-07,  1.0000000e+00]], dtype=float32), array([[ 

FloatProgress(value=0.0, description='Saving', layout=Layout(width='100%'), max=1.0)

Saving time: 0:55:20.335225
Alignment+Saving rate: 0.20 images per second
C:\exiftool\exiftool.exe -csv="\\ds3617xs1\Sensorik\Projektdaten\COBRI\2020\1_SOURCE_DATA\UAV\IFZ\Multispectral\2020_07_28\stacks\log.csv" -overwrite_original \\ds3617xs1\Sensorik\Projektdaten\COBRI\2020\1_SOURCE_DATA\UAV\IFZ\Multispectral\2020_07_28\stacks


0